<a href="https://colab.research.google.com/github/RainaVardhan/DS4002-Project-2/blob/main/SCRIPTS/DS_PROJECT_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/RainaVardhan/DS4002-Project-2.git

Cloning into 'DS4002-Project-2'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 115 (delta 50), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (115/115), 2.69 MiB | 4.43 MiB/s, done.
Resolving deltas: 100% (50/50), done.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

In [3]:
#Loading the dataset from CSV file
df = pd.read_csv('/content/DS4002-Project-2/DATA/1950-2023_all_tornadoes.csv')
df.head(10)

FileNotFoundError: [Errno 2] No such file or directory: '/content/DS4002-Project-2/DATA/1950-2023_all_tornadoes (3).csv'

In [ ]:
#Convert date to datetime and filtered for Minnesota state
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
mn_df = df[df['st'] == 'MN']
mn_df['loss']

In [ ]:
# Create new column using dictionary
mapping = {1.0: 50, 2.0: 300, 3.0: 3000, 4.0: 30000, 5.0: 300000, 6.0: 3000000,
           7.0: 30000000, 8.0: 300000000, 9.0: 5000000000}

# combine this new data with existing DataFrame
mn_df['loss'] = mn_df['loss'].map(mapping).fillna(mn_df['loss'])
mn_df['loss'] = mn_df['loss'] != 0
mn_df

In [ ]:
#Group data by year and aggregate by property loss, crop loss, and tornado frequency
yearly_data = mn_df.groupby(mn_df['date'].dt.year).agg({
    'loss': 'sum',
    'om': 'count'
}).reset_index()

yearly_data.rename(columns={'om': 'num_tornadoes'}, inplace=True)
yearly_data

In [ ]:
#Checking for null values
yearly_data.info()

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(10, 8))

#Plot the number of tornadoes each year
yearly_data['num_tornadoes'].plot(ax=ax[0], title='Number of Tornadoes in Minnesota', color='blue')
ax[0].set_ylabel('Number of Tornadoes')

#Plot the property damage each year
yearly_data['loss'].plot(ax=ax[1], title='Property Damage (in millions)', color='green')
ax[1].set_ylabel('Property Damage')

plt.tight_layout()
plt.show()

In [ ]:
#Perform ADF test to check for stationary of tornado data
result = adfuller(yearly_data['num_tornadoes'])
print(f"ADF Statistic for Number of Tornadoes: {result[0]}")
print(f"p-value: {result[1]}")

In [ ]:
#Applying differencing if series is non-stationary (p-value > 0.05)
if result[1] > 0.05:
    yearly_data['num_tornadoes_diff'] = yearly_data['num_tornadoes'].diff().dropna()
    result = adfuller(yearly_data['num_tornadoes_diff'])
    stationarity_interpretation = "Stationary" if result[1] < 0.05 else "Non-Stationary"

else:
    yearly_data['num_tornadoes_diff'] = yearly_data['num_tornadoes']
    stationarity_interpretation = "Stationary"

print(f"ADF Statistic after differencing: {result[0]}")
print(f"p-value after differencing: {result[1]}")
print(f"Interpretation: The series is {stationarity_interpretation}.")

In [ ]:
max_lags = min(40, len(yearly_data['num_tornadoes']) // 2)
fig, ax = plt.subplots(2, figsize=(12, 8))

#ACF plot to find 'q' (moving average term)
plot_acf(yearly_data['num_tornadoes'], lags=max_lags, ax=ax[0])
ax[0].set_title('ACF Plot (Number of Tornadoes)')

#PACF plot to find 'p' (autoregressive term)
plot_pacf(yearly_data['num_tornadoes'], lags=max_lags, ax=ax[1])
ax[1].set_title('PACF Plot (Number of Tornadoes)')

plt.tight_layout()
plt.show()

In [ ]:
#Fit the ARIME model with parameters p, d, q
model = ARIMA(yearly_data['num_tornadoes_diff'].dropna(), order=(1, 1, 1))
model_fit = model.fit()

print(model_fit.summary())

In [ ]:
#Plot residuals of the fitted model to check if they resemble white noise
residuals = model_fit.resid
plt.figure(figsize=(10, 6))
plt.plot(residuals)
plt.title('Residuals of ARIMA Model')
plt.show()

In [ ]:
#Forecast for the next 10 years based on the fitted ARIMA model
forecast_steps = 10
forecast = model_fit.forecast(steps=forecast_steps)
forecast_years = np.arange(yearly_data.index.max() + 1, yearly_data.index.max() + forecast_steps + 1)

#Plotting historical tornado data and forcasted tornadoes for the next 10 years
plt.figure(figsize=(10, 6))
plt.plot(yearly_data.index, yearly_data['num_tornadoes'], label='Historical Tornadoes')
plt.plot(forecast_years, forecast, label='Forecasted Tornadoes', color='red')
plt.title('Tornado Forecast for the Next 10 Years')
plt.legend()
plt.show()

In [ ]:
#Evaluate the ARIMA model using AIC and BIC metrics
print(f"AIC: {model_fit.aic}")
print(f"BIC: {model_fit.bic}")

In [ ]:
#Split data into training and testing sets for model validation
train_size = int(len(yearly_data) * 0.8)
train, test = yearly_data['num_tornadoes_diff'][:train_size], yearly_data['num_tornadoes_diff'][train_size:]

#Fit the ARIMA model on training data and forecast on the test data
p = 1
d = 1
q = 1
model = ARIMA(train, order=(p, d, q))
model_fit = model.fit()
forecast = model_fit.forecast(steps=len(test))

In [ ]:
#Plot the training data, test data, and forecasted values
plt.figure(figsize=(10, 5))
plt.plot(yearly_data.index[:train_size], train, label='Train', color='blue')
plt.plot(yearly_data.index[train_size:], test, label='Test', color='green')
plt.plot(yearly_data.index[train_size:], forecast, label='Forecast', color='red')
plt.legend()
plt.title('ARIMA Forecast vs Actual')
plt.show()

In [ ]:
#Calculate RMSE to evaluate model's accuracy
rmse = mean_squared_error(test, forecast, squared=False)
print(f"RMSE: {rmse}")

In [ ]:
#Applying differencing if series is non-stationary (p-value > 0.05)
if result[1] > 0.05:
    yearly_data['loss_diff'] = yearly_data['loss'].diff().dropna()
    result = adfuller(yearly_data['loss_diff'])
    stationarity_interpretation = "Stationary" if result[1] < 0.05 else "Non-Stationary"

else:
    yearly_data['loss_diff'] = yearly_data['loss']
    stationarity_interpretation = "Stationary"

print(f"ADF Statistic after differencing: {result[0]}")
print(f"p-value after differencing: {result[1]}")
print(f"Interpretation: The series is {stationarity_interpretation}.")

In [ ]:
max_lags = min(40, len(yearly_data['loss']) // 2)
fig, ax = plt.subplots(2, figsize=(12, 8))

#ACF plot to find 'q' (moving average term)
plot_acf(yearly_data['loss'], lags=max_lags, ax=ax[0])
ax[0].set_title('ACF Plot (Loss)')

#PACF plot to find 'p' (autoregressive term)
plot_pacf(yearly_data['loss'], lags=max_lags, ax=ax[1])
ax[1].set_title('PACF Plot (Loss)')

plt.tight_layout()
plt.show()

In [ ]:
#Fit the ARIME model with parameters p, d, q
model = ARIMA(yearly_data['loss'].dropna(), order=(1, 1, 1))
model_fit = model.fit()

print(model_fit.summary())

In [ ]:
#Forecast for the next 10 years based on the fitted ARIMA model
forecast_steps = 10
forecast = model_fit.forecast(steps=forecast_steps)
forecast_years = np.arange(yearly_data.index.max() + 1, yearly_data.index.max() + forecast_steps + 1)

#Plotting historical tornado data and forcasted tornadoes for the next 10 years
plt.figure(figsize=(10, 6))
plt.plot(yearly_data.index, yearly_data['loss'], label='Past Loss')
plt.plot(forecast_years, forecast, label='Forecasted Loss', color='red')
plt.title('Loss Forecast for the Next 10 Years')
plt.legend()
plt.show()

In [ ]:
#Evaluate the ARIMA model using AIC and BIC metrics
print(f"AIC: {model_fit.aic}")
print(f"BIC: {model_fit.bic}")

In [ ]:
#Split data into training and testing sets for model validation
train_size = int(len(yearly_data) * 0.8)
train, test = yearly_data['loss'][:train_size], yearly_data['loss'][train_size:]

#Fit the ARIMA model on training data and forecast on the test data
p = 1
d = 1
q = 1
model = ARIMA(train, order=(p, d, q))
model_fit = model.fit()
forecast = model_fit.forecast(steps=len(test))

In [ ]:
#Plot the training data, test data, and forecasted values
plt.figure(figsize=(10, 5))
plt.plot(yearly_data.index[:train_size], train, label='Train', color='blue')
plt.plot(yearly_data.index[train_size:], test, label='Test', color='green')
plt.plot(yearly_data.index[train_size:], forecast, label='Forecast', color='red')
plt.legend()
plt.title('ARIMA Forecast vs Actual')
plt.show()

In [ ]:
#Calculate RMSE to evaluate model's accuracy
rmse = mean_squared_error(test, forecast, squared=False)
print(f"RMSE: {rmse}")